# Predicting Flight Delays

The ogjective of this project is to use historical flight information to create a classification model that can return the probability of a flight being delayed. 

We will be looking at data from 2016, 2017, and 2018. All data files are downloaded from OST website, which stores flights on-time performance fro 1987 to present.

**Our MVP**: Our MVP will look to just deliver predictions and exploration on the flight data. Once the MVP is completed, I will look to bring in weather data.


In addition to the data that we are bringing, we will also be using weather data. 

As stated in a published paper by 

> "Our analysis shows that weather shocks like rainfall, snow and wind have a significant impact on departure delays within the U.S. aviation system. Depending on the intensity of the weather shock considered, rain- and snowfall lead to additional departure delays between 10 and 23 min. For wind, the discovered effects are smaller in magnitude, ranging from 1 to 3 min. While the impact of rainfall is more pronounced between April and September, the effect of wind is more evenly distributed across the year. With regard to slow onset weather shocks, our results reveal that cold conditions lead to additional departure delays. For temperatures below 0∘C, our analysis shows an increase in delay on average by around 2 min per departure. For heat, we do not discover any significant amplifying impacts on departure delay. While our results indicate a tendency of higher average departure delays when temperatures go beyond 45∘C, the results are not statistically significant. Considering the recent experiences from Phoenix, where temperatures of 49∘C on June 20th, 2017 lead to the cancellation of more than 40 regional flights, together with the projections by Coffel et al. (2017), we expect extreme temperatures to increase departure delays.8 Our observations for temperatures beyond 45∘C, however, are insufficient for robust estimates."$^1$



$^1$Borsky, S and Unterberger, C (2019) ‘Bad weather and flight delays: The impact of sudden and slow onset weather events’, Economics of Transportation, Volume 18.

The weather data comes from:

Moosavi, Sobhan, Mohammad Hossein Samavatian, Arnab Nandi, Srinivasan Parthasarathy, and Rajiv Ramnath. “Short and Long-term Pattern Discovery Over Large-Scale Geo-Spatiotemporal Data.” In Proceedings of the 25th ACM SIGKDD International Conference on Knowledge Discovery & Data Mining, ACM, 2019.

In [5]:
import pandas as pd
import numpy as np

import os.path
import requests
import io

# Acquistion

In [21]:
# df = pd.read_csv("https://raw.githubusercontent.com/michaelarg/datasets/master/airports.csv")

# airports = df[df.Country == "United States"]

# weather_data = pd.read_csv("US_WeatherEvents_2016-2019.csv")

In [61]:
df = pd.read_csv("2018.csv")

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7213446 entries, 0 to 7213445
Data columns (total 28 columns):
FL_DATE                object
OP_CARRIER             object
OP_CARRIER_FL_NUM      int64
ORIGIN                 object
DEST                   object
CRS_DEP_TIME           int64
DEP_TIME               float64
DEP_DELAY              float64
TAXI_OUT               float64
WHEELS_OFF             float64
WHEELS_ON              float64
TAXI_IN                float64
CRS_ARR_TIME           int64
ARR_TIME               float64
ARR_DELAY              float64
CANCELLED              float64
CANCELLATION_CODE      object
DIVERTED               float64
CRS_ELAPSED_TIME       float64
ACTUAL_ELAPSED_TIME    float64
AIR_TIME               float64
DISTANCE               float64
CARRIER_DELAY          float64
WEATHER_DELAY          float64
NAS_DELAY              float64
SECURITY_DELAY         float64
LATE_AIRCRAFT_DELAY    float64
Unnamed: 27            float64
dtypes: float64(20), int64(3),

Given that there is a large amount of data (+1.5GB), we will filter down to the top 15 largest airports. This will be at least for the exploration phase. We may look to bring additional data once we are ready to model

In [66]:
top_airports = ["ATL", "LAX", "ORD", "DFW", "DEN", "JFK", "SFO", "SEA", "LAS", "MCO", "EWR", "CLT", "PHX", "IAH", "MIA"]

In [72]:
df[df.ORIGIN.str.contains("ATL | LAX")]

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27


In [ ]:
for i in top_airports:
    df.loc[df['ORIGIN'] == i, 'is_top'] = True 
    

In [74]:
df.is_top = df.is_top.fillna(False)

In [75]:
df[df.is_top].ORIGIN.value_counts()

ATL    390046
ORD    332953
DFW    279298
DEN    235989
CLT    233317
LAX    221486
SFO    175849
PHX    173962
IAH    173806
LAS    161113
EWR    143875
SEA    140006
MCO    138296
JFK    129684
MIA     87880
Name: ORIGIN, dtype: int64

In [76]:
df = df[df.is_top]

In [86]:
df.shape

(3017560, 29)

In [88]:
df.isnull().sum()

FL_DATE                      0
OP_CARRIER                   0
OP_CARRIER_FL_NUM            0
ORIGIN                       0
DEST                         0
CRS_DEP_TIME                 0
DEP_TIME                 40437
DEP_DELAY                41790
TAXI_OUT                 41892
WHEELS_OFF               41892
WHEELS_ON                43585
TAXI_IN                  43585
CRS_ARR_TIME                 0
ARR_TIME                 43585
ARR_DELAY                50550
CANCELLED                    0
CANCELLATION_CODE      2975227
DIVERTED                     0
CRS_ELAPSED_TIME             2
ACTUAL_ELAPSED_TIME      49843
AIR_TIME                 49843
DISTANCE                     0
CARRIER_DELAY          2416987
WEATHER_DELAY          2416987
NAS_DELAY              2416987
SECURITY_DELAY         2416987
LATE_AIRCRAFT_DELAY    2416987
Unnamed: 27            3017560
is_top                       0
dtype: int64

We see that even after only keeping the top 15 airports, we stll have a lot of NULL values. Given that at this point we want to know what factors might lead to a delay, we will remove all NULL values for the categorical Dalays. We will still have around 600,000 usable data points.

In [89]:
df = df[df.WEATHER_DELAY.notnull()]

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600573 entries, 11 to 7213429
Data columns (total 29 columns):
FL_DATE                600573 non-null object
OP_CARRIER             600573 non-null object
OP_CARRIER_FL_NUM      600573 non-null int64
ORIGIN                 600573 non-null object
DEST                   600573 non-null object
CRS_DEP_TIME           600573 non-null int64
DEP_TIME               600573 non-null float64
DEP_DELAY              600496 non-null float64
TAXI_OUT               600573 non-null float64
WHEELS_OFF             600573 non-null float64
WHEELS_ON              600573 non-null float64
TAXI_IN                600573 non-null float64
CRS_ARR_TIME           600573 non-null int64
ARR_TIME               600573 non-null float64
ARR_DELAY              600573 non-null float64
CANCELLED              600573 non-null float64
CANCELLATION_CODE      0 non-null object
DIVERTED               600573 non-null float64
CRS_ELAPSED_TIME       600573 non-null float64
ACTUAL_ELA

We see that we largely were able to get rid of any null values. It seems that `Cancelation_code` has no remaining values, we so will drop that column. Additionally, it seems that `Dep_delay` still has some null values, so we will try to impude those.

In [96]:
df.drop(columns = ["Unnamed: 27", "is_top", "CANCELLATION_CODE"], inplace=True)

In [98]:
df[df.DEP_DELAY.isnull()]

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
206834,2018-01-12,9E,3378,ATL,GNV,2230,2230.0,NaN,50.0,2320.0,...,0.0,76.0,99.0,46.0,300.0,0.0,0.0,23.0,0.0,0.0
405036,2018-01-23,9E,4066,ORD,JFK,700,700.0,NaN,49.0,749.0,...,0.0,139.0,163.0,109.0,740.0,0.0,0.0,24.0,0.0,0.0
513786,2018-01-29,9E,3727,ORD,JFK,1334,1334.0,NaN,69.0,1443.0,...,0.0,145.0,189.0,107.0,740.0,0.0,0.0,44.0,0.0,0.0
553576,2018-01-31,9E,4099,JFK,ORD,1926,1926.0,NaN,47.0,2013.0,...,0.0,185.0,201.0,135.0,740.0,0.0,0.0,16.0,0.0,0.0
606927,2018-02-02,9E,4024,JFK,BWI,1709,1709.0,NaN,48.0,1757.0,...,0.0,80.0,98.0,46.0,184.0,0.0,0.0,18.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4470815,2018-08-14,9E,5337,JFK,ORD,1245,1245.0,NaN,56.0,1341.0,...,0.0,167.0,189.0,120.0,740.0,0.0,0.0,22.0,0.0,0.0
4554977,2018-08-18,9E,3386,ATL,HPN,1632,1632.0,NaN,42.0,1714.0,...,0.0,145.0,169.0,124.0,780.0,0.0,0.0,24.0,0.0,0.0
4580696,2018-08-19,9E,3477,ATL,MGM,1801,1801.0,NaN,54.0,1855.0,...,0.0,63.0,97.0,38.0,147.0,0.0,0.0,34.0,0.0,0.0
4670072,2018-08-24,9E,5038,ORD,JFK,1507,1507.0,NaN,17.0,1524.0,...,0.0,153.0,169.0,101.0,740.0,0.0,0.0,16.0,0.0,0.0


We see that the null values seem to indicate there wasn't a delay in take off. We will impude these values with 0

In [105]:
df.DEP_DELAY = df.DEP_DELAY.fillna(0.0)

In [108]:
# Lower case column names

df.columns = map(str.lower, df.columns)

In [110]:
df.isnull().sum()

fl_date                0
op_carrier             0
op_carrier_fl_num      0
origin                 0
dest                   0
crs_dep_time           0
dep_time               0
dep_delay              0
taxi_out               0
wheels_off             0
wheels_on              0
taxi_in                0
crs_arr_time           0
arr_time               0
arr_delay              0
cancelled              0
diverted               0
crs_elapsed_time       0
actual_elapsed_time    0
air_time               0
distance               0
carrier_delay          0
weather_delay          0
nas_delay              0
security_delay         0
late_aircraft_delay    0
dtype: int64

Now that we have the data in the shape that we want it it, we are ready to move onto exploration

# Exploration

We will look to understand if there are any patterns that can help us predict if a flight will be delayed.

## What percentage of flights experience delays?

In [113]:
(df.arr_delay > 0).mean()

1.0

## What airline carrier experiences the most delays?

## What airport experience the most delays?

## What is the most common type of delay?

## Are delays more or less common when the destination is another major airport?